In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da  # import dask.array as da
import dask.bag as db

In [2]:
print(da.stack)

<function stack at 0x7fa23208d8c8>


In [3]:
data = '/Users/melissacirtain/Documents/gmu/AIT726-NLP/Project/data/enron_emails.csv'
df = pd.read_csv(data)
ddf = dd.from_pandas(df, npartitions=10)
ddf


,file,message
npartitions=10,,
0,object,object
51741,...,...
...,...,...
465669,...,...
517400,...,...


In [17]:
type(ddf)
ddf.head(3)
ddf.divisions
ddf.partitions[1]


,file,message
npartitions=1,,
51741,object,object
103482,...,...


In [22]:
ddf.loc[5]['message'].compute()

5    Message-ID: <30965995.1075863688265.JavaMail.e...
Name: message, dtype: object

In [23]:
small_df = ddf.loc[0:100].compute()
small_df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [27]:
email = small_df.loc[1]['message']
email

"Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>\nDate: Fri, 4 May 2001 13:51:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: john.lavorato@enron.com\nSubject: Re:\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nTraveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.\n\nAs far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what 

In [28]:
parsed = pd.DataFrame(columns=['from', 'to', 'date', 'subject', 'mssg'])

In [42]:
email_dict = {}
email_dict['mssg'] = email.split('.pst\n\n')[1]
email_dict['date'] = email.split('\nDate: ')[1].split('\nFrom: ')[0]
email_dict['from'] = email.split(date + '\nFrom: ')[1].split('\nTo: ')[0]
email_dict['to'] = email.split('\nTo: ')[1].split('\nSubject: ')[0]
email_dict['subject'] = email.split('\nSubject: ')[1].split('\n')[0]
email.split('.pst\n\n')[1]
date = email.split('\nDate: ')[1].split('\nFrom: ')[0]
sent_from = email.split(date + '\nFrom: ')[1].split('\nTo: ')[0]
to = email.split('\nTo: ')[1].split('\nSubject: ')[0]
subj = email.split('\nSubject: ')[1].split('\n')[0]

parsed.append(email_dict, ignore_index=True)

,from,to,date,subject,mssg
0,phillip.allen@enron.com,john.lavorato@enron.com,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,Traveling to have a business meeting takes the...


In [53]:
def parse_mssgs(df):
    mssgs = []
    for idx, row in df.iterrows():
        #print(row)
        email = row['message']
        email_dict = {}
        try:
            email_dict['mssg'] = email.split('.pst\n\n')[1]
            email_dict['date'] = email.split('\nDate: ')[1].split('\nFrom: ')[0]
            email_dict['from'] = email.split(date + '\nFrom: ')[1].split('\nTo: ')[0]
            email_dict['to'] = email.split('\nTo: ')[1].split('\nSubject: ')[0]
            email_dict['subject'] = email.split('\nSubject: ')[1].split('\n')[0]
            mssgs.append(email_dict)
        except Exception as e:
            print(f'could not parse message idx {idx}: \n{e}')

        print(email_dict)
    df = pd.DataFrame(mssgs)
    print('created df of shape', df.shape)
    return df

parse_mssgs(small_df)


could not parse message idx 0: 
list index out of range
{'mssg': 'Here is our forecast\n\n ', 'date': 'Mon, 14 May 2001 16:39:00 -0700 (PDT)'}
{'mssg': "Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.\n\nAs far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.  \n\nMy suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time.\n", 'date': 'Fri, 4 May 2001 13:51:00 -0700 (PDT)', 'from

,mssg,date,from,to,subject
0,Traveling to have a business meeting takes the...,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:


### Using a kaggle preprocessing notebook
https://www.kaggle.com/code/oalvay/enron-emails-complete-preprocessing/notebook

In [2]:
import numpy as np
import pandas as pd

import os, gc, re

#df = pd.read_csv('/kaggle/input/enron-email-dataset/emails.csv')
#df.head(5)

In [3]:
newdf = pd.read_csv('/Users/melissacirtain/Downloads/enron_05_17_2015_with_labels_v2.csv_enron_05_17_2015_with_labels_v2.csv')

/Users/melissacirtain/work/envs/ml-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
newdf.shape

(517401, 52)

In [5]:
newdf.head()

,Unnamed: 0,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,...,Cat_10_level_1,Cat_10_level_2,Cat_10_weight,Cat_11_level_1,Cat_11_level_2,Cat_11_weight,Cat_12_level_1,Cat_12_level_2,Cat_12_weight,labeled
0,0,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,1,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),Re: test,Phillip K Allen,Leah Van Arsdall,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,3,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,Phillip K Allen,Randall L Gay,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,4,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),Re: Hello,Phillip K Allen,Greg Piper,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [6]:
newdf.columns

Index(['Unnamed: 0', 'Message-ID', 'Date', 'From', 'To', 'Subject', 'X-From',
       'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName',
       'content', 'user', 'Cat_1_level_1', 'Cat_1_level_2', 'Cat_1_weight',
       'Cat_2_level_1', 'Cat_2_level_2', 'Cat_2_weight', 'Cat_3_level_1',
       'Cat_3_level_2', 'Cat_3_weight', 'Cat_4_level_1', 'Cat_4_level_2',
       'Cat_4_weight', 'Cat_5_level_1', 'Cat_5_level_2', 'Cat_5_weight',
       'Cat_6_level_1', 'Cat_6_level_2', 'Cat_6_weight', 'Cat_7_level_1',
       'Cat_7_level_2', 'Cat_7_weight', 'Cat_8_level_1', 'Cat_8_level_2',
       'Cat_8_weight', 'Cat_9_level_1', 'Cat_9_level_2', 'Cat_9_weight',
       'Cat_10_level_1', 'Cat_10_level_2', 'Cat_10_weight', 'Cat_11_level_1',
       'Cat_11_level_2', 'Cat_11_weight', 'Cat_12_level_1', 'Cat_12_level_2',
       'Cat_12_weight', 'labeled'],
      dtype='object')

In [16]:
for x in range(24,27):
    print(newdf.loc[x, 'content'])
    print()



Liane, As we discussed yesterday, I am concerned there has been an attempt to manipulate the El Paso San Juan monthly index. A single buyer entered the marketplace on both September 26 and 27 and paid above market prices ($4.70-$4.80) for San Juan gas with the intent to distort the index. At the time of these trades, offers for physical gas at significantly (10 to 15 cents) lower prices were bypassed in order to establish higher trades to report into the index calculation. Additionally, these trades are out of line with the associated financial swaps for San Juan. We have compiled a list of financial and physical trades executed from September 25 to September 27. These are the complete list of trades from Enron Online (EOL), Enron's direct phone conversations, and three brokerage firms (Amerex, APB, and Prebon). Please see the attached spreadsheet for a trade by trade list and a summary. We have also included a summary of gas daily prices to illustrate the value of San Juan based on se

In [15]:
for x in range(100):
    print(x, newdf.loc[x, 'Subject'])
    print()


0 nan

1 Re:

2 Re: test

3 nan

4 Re: Hello

5 Re: Hello

6 nan

7 Re: PRC review - phone calls

8 Re: High Speed Internet Access

9 FW: fixed forward or other Collar floor gas price terms

10 Re: FW: fixed forward or other Collar floor gas price terms

11 nan

12 Consolidated positions: Issues & To Do list

13 Consolidated positions: Issues & To Do list

14 nan

15 Re: 2001 Margin Plan

16 Var, Reporting and Resources Meeting

17 nan

18 Westgate

19 Meeting re: Storage Strategies in the West

20 nan

21 Re: Not business related..

22 Re: Original Sept check/closing

23 San Juan Index

24 San Juan Index

25 Investment Structure

26 Investment Structure

27 nan

28 nan

29 Re: Gas Trading Vision meeting

30 nan

31 Gas Physical/Financial Position

32 closing

33 nan

34 nan

35 nan

36 nan

37 nan

38 nan

39 nan

40 nan

41 Westgate Proforma-Phillip Allen.xls

42 nan

43 nan

44 Re: burnet

45 nan

46 Re: Sept 1 Payment

47 nan

48 Contact list for mid market

49 nan

50 Re: 2- SURVE